# Feed forward neural network to predict price per night

In [14]:
import json
import yaml
import torch
import torch.nn as nn
import pandas as pd
import plotly.express as px

### Load best model

In [15]:
class FeedforwardNeuralNetModel(nn.Module):
    # Initialize the layers
    def __init__(self, input_dim, hidden_dim_array, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        self.hidden_dim_array = hidden_dim_array
        self.layers = nn.ModuleList()
        for i in range(len(hidden_dim_array)):
            self.layers.append(nn.Linear(input_dim, hidden_dim_array[i]))
            input_dim = hidden_dim_array[i]  # For the next layer
            self.layers.append(nn.ReLU())  # Activation function
        self.layers.append(nn.Linear(input_dim, output_dim))
    
    # Perform the computation
    def forward(self, x):
        output = x
        for layer in self.layers:
            output = layer(output)
        return output

model = torch.load('trains/2022-10-21_10:31:14/model.pt')
with open('trains/2022-10-21_10:31:14/hyperparameters.yml', 'r') as file:
    params = yaml.load(file, Loader=yaml.Loader)
with open('trains/2022-10-21_10:31:14/metrics.json', 'r') as file:
    metrics = json.load(file)

print(f'Parameters: {params}\n\nMetrics: {metrics}')

Parameters: {'hidden_dim_array': [4, 2], 'learning_rate': 1e-05, 'optimiser': <class 'torch.optim.sgd.SGD'>}

Metrics: {'Training time': 49.920105934143066, 'Inference latency': 0.00014894188748545859, 'Training RMSE': 102.57119750976562, 'Training R2 score': 0.3379220962524414, 'Validation RMSE': 104.76048278808594, 'Validation R2 score': 0.3601261377334595, 'Test RMSE': 98.00000762939453, 'Test R2 score': 0.39434707164764404}


In [16]:
dict = {'Train': {'RMSE': metrics['Training RMSE'], 'R2': metrics['Training R2 score']}, 'Test': {'RMSE': metrics['Test RMSE'], 'R2': metrics['Test R2 score']}}
df = pd.DataFrame(dict).transpose()

rmse_bar = px.bar(
    df['RMSE'],
    title='Best model RMSE',
    labels={'index': '', 'value': 'RMSE'},
    template='plotly_dark',
)
rmse_bar.update_layout(showlegend=False)
rmse_bar.update_yaxes(range=[60, 110])
rmse_bar.write_image('../../../../README-images/nn-regression-train-test-rmse.png', scale=20)
rmse_bar.show()

r2_bar = px.bar(
    df['R2'],
    title='Best model R2 score',
    labels={'index': '', 'value': 'R2 score'},
    template='plotly_dark',
)
r2_bar.update_layout(showlegend=False)
r2_bar.write_image('../../../../README-images/nn-regression-train-test-r2.png', scale=20)
r2_bar.show()

